In [1]:
#pip install pymssql

In [1]:
import pandas as pd
import pymssql
from sqlalchemy import create_engine

In [2]:
def connDatabase():
    conn = pymssql.connect(
        host='linyang5124.database.windows.net',
        user='linyang',
        password='Techlent2022',
        database='lin',
        charset='utf8'
        )
    cursor = conn.cursor(as_dict=True)
    return cursor

In [3]:
cursor = connDatabase()

In [6]:
def checkRedeemStatus(trains):
    for i in range(len(trains)):
        train = trains.iloc[i]
        sql_1 = 'select * from coupon_item_mapping where coupon_id={0}'.format(train['coupon_id'])
        print(sql_1)

        cursor.execute(sql_1)
        results = cursor.fetchall()
        items = pd.DataFrame(data = results)
        item_ids = ''
        for item in items['item_id']:
            item_ids = item_ids + str(item) + ','
        item_ids = item_ids.strip(',')
        sql_2 = 'select * from customer_transaction_data where item_id in ({0})'.format(item_ids)
        print(sql_2)

        sql_3 = 'select * from campaign_data where campaign_id={0}'.format(train['campaign_id'])
        cursor.execute(sql_3)
        results = cursor.fetchall()
        campaign = pd.DataFrame(data = results)
        start_date = campaign.iloc[0]['start_date']
        end_date = campaign.iloc[0]['end_date']

        sql_4 = 'select * from customer_transaction_data \
                    where item_id in ({0}) and date>=\'{1}\' \
                    and date<=\'{2}\' and customer_id={3}'.format(item_ids, start_date, end_date, train['customer_id'])
        print(sql_4)
        cursor.execute(sql_4)
        results = cursor.fetchall()
        transaction = pd.DataFrame(data = results)
        if len(transaction)==0: 
            redemption_status = False
        else:
            if sum(transaction['coupon_discount']) < 0:
                   redemption_status = True
            else:
                   redemption_status = False

        if redemption_status == train['redemption_status']:
            print('id:{0}, campaign_id:{1}, coupon_id:{2}, customer_id:{3}, redemption_status:{4} is matched.'.
                  format(train['id'],train['campaign_id'],train['coupon_id'],train['customer_id'], train['redemption_status']))     
        else:
            print('******id:{0}, campaign_id:{1}, coupon_id:{2}, customer_id:{3}, redemption_status:{4} is NOT matched.'.
                  format(train['id'],train['campaign_id'],train['coupon_id'],train['customer_id'], train['redemption_status']))       

In [7]:
#sql_0 = 'select top 10 * from train order by redemption_status desc'
sql_0 = 'select top 1 * from train where redemption_status=1'
cursor.execute(sql_0)
results = cursor.fetchall()
trains = pd.DataFrame(data = results)

checkRedeemStatus(trains)

select * from coupon_item_mapping where coupon_id=413
select * from customer_transaction_data where item_id in (37212,24453,5977,56675,43462,56404,56554,56406,56407,65741,65765,31642,8792)
select * from customer_transaction_data                     where item_id in (37212,24453,5977,56675,43462,56404,56554,56406,56407,65741,65765,31642,8792) and date>='2013-05-19'                     and date<='2013-07-05' and customer_id=276
id:72, campaign_id:13, coupon_id:413, customer_id:276, redemption_status:True is matched.
